# 1. Import libraries


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
#!pip install fer
from fer import FER
#!pip install mtcnn
from mtcnn import MTCNN
import cv2

# 2. Link to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Paths

In [ ]:
# path to file folder
path_csv_out = "/content/drive/My Drive/Thesis/"

# path to image folder
path_images = "/content/drive/My Drive/Thesis/Images"

# get list of image filenames
image_filenames = os.listdir(path_images)

# print number of images
print(f"Number of images: {len(image_filenames)}")


Number of images: 149


# 4. Model for emotion detection

In [ ]:
emotion_detector = FER(mtcnn=True)

TypeError: MTCNN.detect_faces() missing 1 required positional argument: 'img'

# 5. Example

In [ ]:
test_img = path_images + "/id_34242.jpg"
analysis = emotion_detector.detect_emotions(test_img)
print(analysis)

img = cv2.cvtColor(cv2.imread(test_img), cv2.COLOR_BGR2RGB)
detections = detector.detect_faces(img)
detections

[{'box': [89, 39, 264, 324], 'emotions': {'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happy': 1.0, 'sad': 0.0, 'surprise': 0.0, 'neutral': 0.0}}, {'box': [4, 149, 34, 41], 'emotions': {'angry': 0.14, 'disgust': 0.01, 'fear': 0.17, 'happy': 0.25, 'sad': 0.14, 'surprise': 0.02, 'neutral': 0.27}}]
1/1 [==============================] - 0s 42ms/step


[{'box': [80, 20, 280, 352],
  'confidence': 0.998687744140625,
  'keypoints': {'left_eye': (183, 133),
   'right_eye': (302, 185),
   'nose': (225, 219),
   'mouth_left': (136, 240),
   'mouth_right': (257, 291)}},
 {'box': [395, 0, 243, 233],
  'confidence': 0.9982835054397583,
  'keypoints': {'left_eye': (472, 55),
   'right_eye': (581, 42),
   'nose': (533, 116),
   'mouth_left': (483, 168),
   'mouth_right': (574, 163)}}]

# 6. Multi-image analysis

In [ ]:
# create an empty list to store the results
results = []

# iterate over all the image files in the directory
for filename in image_filenames:
    # create the full path to the image file
    image = os.path.join(path_images, filename)

    emotion_detector = FER(mtcnn=True)

    # detect emotions
    analysis = emotion_detector.detect_emotions(image)

    # detect dominant emotion
    dominant_emotion, emotion_score = emotion_detector.top_emotion(image)

    # if there are no faces detected, create a default result with NaN values
    if not analysis:
        result = {'filename': filename, 'name': filename.split(sep='.')[0]}
        result.update({'angry': np.NaN, 'disgust': np.NaN, 'fear': np.NaN, 'happy': np.NaN, 'sad': np.NaN, 'surprise': np.NaN, 'neutral': np.NaN})
        result['dominant_emotion'] = np.NaN
    else:
        # get the emotions dictionary from the analysis
        emotions = analysis[0]['emotions']

        # create a new dictionary with the filename and emotions
        result = {'filename': filename, 'name': filename.split(sep='.')[0]}
        result.update(emotions)
        result['dominant_emotion'] = dominant_emotion  # add dominant emotion column

    # add the result to the list
    results.append(result)

# create a dataframe from the results
df = pd.DataFrame(results)
df

,filename,name,angry,disgust,fear,happy,sad,surprise,neutral,dominant_emotion
0,id_30054.jpg,id_30054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id_30103.jpg,id_30103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id_30104.jpeg,id_30104,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
3,id_30105.jpg,id_30105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id_30107.jpg,id_30107,0.01,0.0,0.00,0.94,0.00,0.05,0.00,happy
...,...,...,...,...,...,...,...,...,...,...
144,id_34239.jpg,id_34239,0.07,0.0,0.01,0.08,0.35,0.00,0.49,neutral
145,id_34242.jpg,id_34242,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
146,id_34248.jpg,id_34248,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
147,id_34250.jpg,id_34250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 7. Merge data

In [ ]:
# create a df with the filenames list
df_name = pd.DataFrame(columns=['uid','name'])
i = 0
for filename in image_filenames:
  name = filename.split(sep='.')[0]
  uid = name.split(sep='_')[1]
  df_name.loc[i,'name'] = name
  df_name.loc[i,'uid'] = uid
  i = i + 1
df_name

,uid,name
0,30054,id_30054
1,30103,id_30103
2,30104,id_30104
3,30105,id_30105
4,30107,id_30107
...,...,...
144,34239,id_34239
145,34242,id_34242
146,34248,id_34248
147,34250,id_34250


In [ ]:
# merge df_name with df on 'name'
df_merged = df_name.merge(df, how='left', on='name')
df_merged

,uid,name,filename,angry,disgust,fear,happy,sad,surprise,neutral,dominant_emotion
0,30054,id_30054,id_30054.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30103,id_30103,id_30103.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30104,id_30104,id_30104.jpeg,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
3,30105,id_30105,id_30105.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30107,id_30107,id_30107.jpg,0.01,0.0,0.00,0.94,0.00,0.05,0.00,happy
...,...,...,...,...,...,...,...,...,...,...,...
144,34239,id_34239,id_34239.jpg,0.07,0.0,0.01,0.08,0.35,0.00,0.49,neutral
145,34242,id_34242,id_34242.jpg,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
146,34248,id_34248,id_34248.jpg,0.00,0.0,0.00,1.00,0.00,0.00,0.00,happy
147,34250,id_34250,id_34250.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# add image data to the csv file
df_camp = pd.read_csv ("/content/drive/My Drive/Thesis/crowdfunding_data.csv")
df_camp['uid'] = df_camp['uid'].apply(str)
df_analysis = df_merged.merge(df_camp, how='left', on='uid')
df_analysis
df_analysis.to_csv( path_csv_out + "full.csv",sep=",")